In [1]:
import sys

sys.path.insert(0, "..")
sys.path.insert(0, "../src")

In [2]:
from src.constants import *
from src.models import SimpleICL
from src.dataset import get_data_loader
from src.utils import iterate_models, load_config, parse_dict

import dill
import jax
import jax.numpy as jnp
import jax.random as jrandom
import numpy as np
import os

In [3]:
# base_path = "/Users/chanb/research/ualberta/icl/cc_results/paper_experiments/simple_icl_models/"
# alpha_path = os.path.join(
#     base_path,
#     "alpha",
#     # lr 1e-5
#     # "dataset_size_1024-p_relevant_context_0.9-num_low_prob_classes_45-seed_0-10-05-24_11_53_02-a0b2cf45-ac18-40c0-a721-e345192e236b"
#     # lr 3e-4
#     "dataset_size_1024-p_relevant_context_0.9-num_low_prob_classes_45-seed_0-10-04-24_09_03_44-b012b211-2c20-4dab-a101-0007e39d152e"
# )

# ic_path = os.path.join(
#     base_path,
#     "ic",
#     "dataset_size_1024-p_relevant_context_0.9-num_low_prob_classes_45-seed_0-10-03-24_18_15_13-8b7969cc-ae13-4aee-a69f-fb81544f8c3b/models/1000000.dill"
# )

# iw_path = os.path.join(
#     base_path,
#     "iw",
#     "dataset_size_1024-p_relevant_context_0.9-num_low_prob_classes_45-seed_0-10-03-24_17_49_12-986a33eb-d0dc-4283-a6fc-5072a98c886a/models/1000000.dill"
# )

alpha_path = "/Users/chanb/research/ualberta/icl/cc_results/paper_experiments/simple_icl_models/dataset_size_1024-p_relevant_context_0.9-num_low_prob_classes_45-seed_0-10-06-24_16_14_02-be34d5b6-3315-4102-ad0e-7dee7666e60e/"
iw_path = ic_path = None

In [4]:
config_dict, config = load_config(alpha_path)
config_dict["num_workers"] = 1
config_dict["dataset_kwargs"]["train"] = False
config = parse_dict(config_dict)

In [ ]:
config

In [ ]:
train_ds, train_dataset = get_data_loader(
    config,
)

In [7]:
batch = next(train_ds)

In [8]:
model = SimpleICL(output_dim=50, num_contexts=1, load_iw=iw_path, load_ic=ic_path)

In [9]:
params = model.init(
    jrandom.PRNGKey(config.seeds.learner_seed),
    train_dataset.input_space,
    train_dataset.output_space,
)

In [10]:
alpha_params = dill.load(open(
    os.path.join(alpha_path, "models", "1000000.dill"),
    "rb",
))

In [11]:
# params["alpha"] = alpha_params["model"]["alpha"]
params = alpha_params["model"]

In [ ]:
params["ic_predictor"]

In [13]:
from src.utils import iterate_models

In [ ]:
params[CONST_MODEL]["ic_predictor"]["ic_predictor"].keys()

In [ ]:
for params, model, checkpoint_i in iterate_models(alpha_path):
    print(params.keys(), model, checkpoint_i)
    print(params[CONST_MODEL]["ic_predictor"].keys())
    model.ic_predictor.forward(params[CONST_MODEL]["ic_predictor"], batch)

In [ ]:
model.ic_predictor.forward(params["ic_predictor"], batch)

In [17]:
log_probs, aux = model.forward(params, batch, eval=True)

In [ ]:
np.vstack((
    np.argmax(batch["target"][:, -1], axis=-1),
    np.argmax(aux["ic_pred"], axis=-1),
    np.argmax(aux["iw_pred"], axis=-1),
    aux["p_iwl"].flatten()
)).T